In [ ]:
# !pip install crewai

In [ ]:
import os
os.environ["SERPER_API_KEY"] = ""  # serper.dev API key
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"  # Example model


In [ ]:
from crewai import Agent
from crewai_tools import SerperDevTool

search_tool = SerperDevTool()

llm_model = 'gpt-3.5-turbo'

researcher = Agent(
    role='Senior Researcher',
    goal='Uncover groundbreaking technologies in {topic}',
    verbose=True,
    memory=True,
    backstory=(
        "Driven by curiosity, you're at the forefront of "
        "innovation, eager to explore and share knowledge that could change "
        "the world."
    ),
    tools=[search_tool],
    allow_delegation=True
)

writer = Agent(
    role='Writer',
    goal='Narrate compelling tech stories about {topic}',
    verbose=True,
    memory=True,
    backstory=(
        "With a flair for simplifying complex topics, you craft "
        "engaging narratives that captivate and educate, bringing new "
        "discoveries to light in an accessible manner."
    ),
    tools=[search_tool],
    allow_delegation=False
)


In [ ]:
from crewai import Task

research_task = Task(
    description=(
        "Identify the current state in {topic}. "
        "Focus on identifying pros and cons and the overall narrative. "
        "Your final report should clearly articulate the key points, "
        "its market opportunities, and potential risks. Do not focus on future predictions. Instead, focus on the impact that's visible today."
    ),
    expected_output='A comprehensive 3-paragraph report on the latest AI trends.',
    tools=[search_tool],
    agent=researcher
)

write_task = Task(
    description=(
        "Compose an insightful article on {topic}. "
        "Focus on the latest trends and how it's impacting the industry. Do not focus on future predictions. Instead, focus on the impact that's visible today."
        "This article should be easy to understand, engaging, and positive."
    ),
    expected_output='A 4-paragraph article on {topic} advancements formatted as markdown.',
    tools=[search_tool],
    agent=writer,
    async_execution=False,
    output_file='new-blog-post.md'  # Example of output customization
)


In [ ]:
from crewai import Crew, Process

crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, write_task],
    process=Process.sequential  # Optional: Sequential task execution is default
)

In [ ]:
result = crew.kickoff(inputs={'topic': 'AI agents in production environments'})
print(result)